In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.precision", 4)

import warnings
warnings.filterwarnings("ignore")

In [2]:
hmoty = 4 # pocet hmot
tvar = 1 # vypocet pre tvar
M = 1500 # kg

In [3]:
delta11 = 0.30367329
delta12 = 0.19217220
delta13 = 0.09490424
delta14 = 0.02610248
delta21 = 0.19217220
delta22 = 0.12812319
delta23 = 0.06643804
delta24 = 0.01898592
delta31 = 0.09490424
delta32 = 0.06643804
delta33 = 0.03797184
delta34 = 0.01186937
delta41 = 0.02610248
delta42 = 0.01898592
delta43 = 0.01186937
delta44 = 0.00475282

In [4]:
delta = np.array([
    [delta11, delta12, delta13, delta14],
    [delta21, delta22, delta23, delta24],
    [delta31, delta32, delta33, delta34],
    [delta41, delta42, delta43, delta44]])
delta

array([[0.30367329, 0.1921722 , 0.09490424, 0.02610248],
       [0.1921722 , 0.12812319, 0.06643804, 0.01898592],
       [0.09490424, 0.06643804, 0.03797184, 0.01186937],
       [0.02610248, 0.01898592, 0.01186937, 0.00475282]])

In [5]:
def delta_inv(delta):
    return np.linalg.inv(delta)

In [6]:
delta_inv_ = delta_inv(delta)
delta_inv_

array([[ 112.70242071, -255.55984283,  181.56949452,  -51.52508671],
       [-255.55984283,  692.68626882, -670.21480744,  310.2341621 ],
       [ 181.56949452, -670.21480744, 1002.90384938, -824.4802549 ],
       [ -51.52508671,  310.2341621 , -824.4802549 , 1313.09680685]])

In [7]:
x = np.linalg.eig(delta_inv_) # eigenvalues and eigenvectors of delta_inv
z = x[0] # eigenvalues
reoder = [z[2], z[3], z[1], z[0]] # reorder eigenvalues
L = np.diag(reoder) # diagonal matrix of eigenvalues
L

array([[2.16655965e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 8.98501754e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 7.15250120e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.31412249e+03]])

In [8]:
mm = np.diag([M, M, M, M])
mm

array([[1500,    0,    0,    0],
       [   0, 1500,    0,    0],
       [   0,    0, 1500,    0],
       [   0,    0,    0, 1500]])

In [9]:
omega = np.nan_to_num(np.sqrt(L/mm))
omega

array([[0.03800491, 0.        , 0.        , 0.        ],
       [0.        , 0.244745  , 0.        , 0.        ],
       [0.        , 0.        , 0.69053125, 0.        ],
       [0.        , 0.        , 0.        , 1.2420742 ]])

In [10]:
V0k = np.zeros((hmoty, hmoty-1))

In [11]:
# diag matrix of hmoty * omega
p4 = np.eye(hmoty) * omega[tvar-1, tvar-1]
p4

array([[0.03800491, 0.        , 0.        , 0.        ],
       [0.        , 0.03800491, 0.        , 0.        ],
       [0.        , 0.        , 0.03800491, 0.        ],
       [0.        , 0.        , 0.        , 0.03800491]])

In [12]:
p5 = delta_inv_ - mm * p4**2
p5

array([[ 110.53586107, -255.55984283,  181.56949452,  -51.52508671],
       [-255.55984283,  690.51970917, -670.21480744,  310.2341621 ],
       [ 181.56949452, -670.21480744, 1000.73728973, -824.4802549 ],
       [ -51.52508671,  310.2341621 , -824.4802549 , 1310.9302472 ]])

In [14]:
# drop first row and first column
A = p5[1:hmoty, 1:hmoty]
# get first column and drop first row
B = p5[1:hmoty, 0:1]

In [15]:
S = np.linalg.solve(A, B)*-1 # -1 lebo som prehodil posledny stlpec na druhu stranu rovnice
S = np.insert(S,0,1)
S

array([1.        , 0.6469836 , 0.32811761, 0.09255654])

In [18]:
# insert S to V0k
V0k[:, 0] = S
V0k

array([[1.        , 0.        , 0.        ],
       [0.6469836 , 0.        , 0.        ],
       [0.32811761, 0.        , 0.        ],
       [0.09255654, 0.        , 0.        ]])

In [19]:
# dot product of V0k transposed, mm and V0k
q0 = 1/np.sqrt(np.dot(np.dot(V0k.T, mm),V0k))[tvar-1 ,tvar-1]
q0

0.020841369806031143

In [24]:
V0 = q0*S*10**3
print(f'vlastny tvar {tvar} {V0}')

vlastny tvar 1 [20.84136981 13.48402444  6.83842055  1.92900506]
